<a href="https://colab.research.google.com/github/0Nguyen0Cong0Tuan0/Road-Buddy-Challenge/blob/main/models/yolo_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YOLO11 Fine-tuning for Traffic Object Detection**


This notebook **fine-tunes YOLO11n and YOLO11l** on custom traffic datasets to improve detection of **road objects** such as cars, trucks, buses, lanes, traffic lights, road signs and **exclude unrelated** such as toothbrush, skis, wine glass, etc.



**Datasets**

| Dataset | Classes | Focus |
|---------|---------|-------|
| BDD100K | 12 | Vehicles, pedestrians, traffic signs/lights |
| Road Lane v2 | 6 | Lane line types (dotted, solid, divider, etc.) |


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Setup & Installation**

In [13]:
!pip install ultralytics -q

In [14]:
import ultralytics
import os
import yaml
import time
from collections import Counter
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import shutil
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'models' else Path.cwd()
DATA_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'traffic datasets'
MODELS_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'models'
RUNS_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'runs'

# Dataset paths
DATASETS = {
    'bdd100k': {
        'path': DATA_DIR / 'bdd100k',
        'yaml': DATA_DIR / 'bdd100k' / 'data_yolo.yaml',
        'description': 'BDD100K - Vehicles, pedestrians, traffic objects',
        'classes': ['car', 'truck', 'bus', 'train', 'person', 'rider',
                   'bike', 'motor', 'traffic light', 'traffic sign', 'lane', 'drivable area']
    },
    'road_lane': {
        'path': DATA_DIR / 'Road Lane.v2i.yolo26',
        'yaml': DATA_DIR / 'Road Lane.v2i.yolo26' / 'data.yaml',
        'description': 'Road Lane v2 - Lane line types',
        'classes': ['divider-line', 'dotted-line', 'double-line',
                   'random-line', 'road-sign-line', 'solid-line']
    }
}

for name, info in DATASETS.items():
    exists = info['path'].exists()
    yaml_exists = info['yaml'].exists()
    print(f"{name}: {info['path'].name}")
    print(f"   YAML: {info['yaml'].name} ({'found' if yaml_exists else 'NOT FOUND'})")
    print(f"   Classes: {len(info['classes'])} - {info['classes'][:5]}...")

bdd100k: bdd100k
   YAML: data_yolo.yaml (found)
   Classes: 12 - ['car', 'truck', 'bus', 'train', 'person']...
road_lane: Road Lane.v2i.yolo26
   YAML: data.yaml (found)
   Classes: 6 - ['divider-line', 'dotted-line', 'double-line', 'random-line', 'road-sign-line']...


In [16]:
# Training configuration
TRAINING_CONFIG = {
    'epochs': 50,          # Number of training epochs
    'imgsz': 640,          # Image size
    'batch': 16,           # Batch size (reduce if GPU OOM)
    'patience': 10,        # Early stopping patience
    'device': DEVICE,      # Training device
    'workers': 4,          # Data loader workers
    'save': True,          # Save checkpoints
    'plots': True,         # Generate training plots
    'verbose': True,       # Verbose output
}


print("\nTraining Configuration")

for key, value in TRAINING_CONFIG.items():
    print(f"   {key}: {value}")


Training Configuration
   epochs: 50
   imgsz: 640
   batch: 16
   patience: 10
   device: cuda
   workers: 4
   save: True
   plots: True
   verbose: True


## **Fine-tuning on Road Lane Dataset**

### **Train YOLO11n on Road Lane**

In [19]:
model_n_lane = YOLO('yolo11n.pt')
model_n_lane.info()

YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(181, 2624080, 0, 6.614336)

In [20]:
print("Fine-tuning YOLO11n on Road Lane Dataset")

# Start training
results_n_lane = model_n_lane.train(
    data=str(DATASETS['road_lane']['yaml']),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11n_road_lane',
    exist_ok=True,
    **TRAINING_CONFIG
)

Fine-tuning YOLO11n on Road Lane Dataset
Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/traffic datasets/Road Lane.v2i.yolo26/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=yolo11n_road_lane, nbs=64, nms=False, opset=None, op

### **Train YOLO11l on Road Lane**

In [21]:
model_l_lane = YOLO('yolo11l.pt')
model_l_lane.info()

YOLO11l summary: 357 layers, 25,372,160 parameters, 0 gradients, 87.6 GFLOPs


(357, 25372160, 0, 87.6134912)

In [22]:
# Reduce batch size for larger model
config_l = TRAINING_CONFIG.copy()
config_l['batch'] = 8

results_l_lane = model_l_lane.train(
    data=str(DATASETS['road_lane']['yaml']),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11l_road_lane',
    exist_ok=True,
    **config_l
)

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/traffic datasets/Road Lane.v2i.yolo26/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=yolo11l_road_lane, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask

## **Fine-tuning on BDD100K Dataset**

In [27]:
bdd_labels_dir = DATASETS['bdd100k']['path'] / 'yolo_labels'

if bdd_labels_dir.exists():
    train_labels = list((bdd_labels_dir / 'train').glob('*.txt')) if (bdd_labels_dir / 'train').exists() else []
    val_labels = list((bdd_labels_dir / 'val').glob('*.txt')) if (bdd_labels_dir / 'val').exists() else []
    print(f"BDD100K YOLO labels found:")
    print(f"   Train labels: {len(train_labels)}")
    print(f"   Val labels: {len(val_labels)}")
else:
    print("BDD100K YOLO labels not found!")

BDD100K YOLO labels found:
   Train labels: 51166
   Val labels: 10000


In [31]:
# Fine-tune YOLO11n on BDD100K
model_n_bdd = YOLO('yolo11n.pt')

results_n_bdd = model_n_bdd.train(
    data=str(DATASETS['bdd100k']['yaml']),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11n_bdd100k',
    exist_ok=True,
    **TRAINING_CONFIG
)

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/traffic datasets/bdd100k/data_yolo.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=yolo11n_bdd100k, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

KeyboardInterrupt: 

In [ ]:
model_l_bdd = YOLO('yolo11l.pt')

config_l = TRAINING_CONFIG.copy()
config_l['batch'] = 8

results_l_bdd = model_l_bdd.train(
    data=str(DATASETS['bdd100k']['yaml']),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11l_bdd100k',
    exist_ok=True,
    **config_l
)

## **Load Fine-tuned Models for Evaluation**

In [ ]:
def find_best_weights(run_dir):
    """Find the best.pt weights file in a training run directory."""
    weights_path = run_dir / 'weights' / 'best.pt'
    if weights_path.exists():
        return weights_path
    return None

# Collect all trained models
FINETUNED_MODELS = {}

model_runs = [
    ('YOLO11n_RoadLane', RUNS_DIR / 'finetune' / 'yolo11n_road_lane'),
    ('YOLO11l_RoadLane', RUNS_DIR / 'finetune' / 'yolo11l_road_lane'),
    ('YOLO11n_BDD100K', RUNS_DIR / 'finetune' / 'yolo11n_bdd100k'),
    ('YOLO11l_BDD100K', RUNS_DIR / 'finetune' / 'yolo11l_bdd100k'),
]

for name, run_dir in model_runs:
    weights = find_best_weights(run_dir)
    if weights:
        try:
            model = YOLO(weights)
            FINETUNED_MODELS[name] = {
                'model': model,
                'path': weights,
                'classes': list(model.names.values())
            }
            print(f"{name}: Loaded ({len(model.names)} classes)")
        except Exception as e:
            print(f"{name}: Failed to load - {e}")
    else:
        print(f"{name}: Not trained yet")

In [ ]:
PRETRAINED_MODELS = {
    'YOLO11n_Pretrained': YOLO('yolo11n.pt'),
    'YOLO11l_Pretrained': YOLO('yolo11l.pt'),
}

print("Pre-trained models:")
for name, model in PRETRAINED_MODELS.items():
    print(f"   {name}: {len(model.names)} classes (COCO)")

## **Performance evaluation**

Compare fine-tuned models against pre-trained models.

### **Inference Speed Test**

In [ ]:
def benchmark_inference(model, images, model_name, num_runs=3, warmup=2):
    """
    Benchmark inference speed of a model.
    """
    times = []
    detections = []
    confidences = []
    class_counts = Counter()

    # Warmup
    for _ in range(warmup):
        if images:
            _ = model(images[0], verbose=False)

    # Benchmark
    for _ in range(num_runs):
        for img in images:
            start = time.time()
            results = model(img, verbose=False)
            inference_time = (time.time() - start) * 1000
            times.append(inference_time)

            for r in results:
                n_det = len(r.boxes)
                detections.append(n_det)
                if n_det > 0:
                    confs = r.boxes.conf.cpu().numpy()
                    confidences.extend(confs.tolist())
                    for cls_id in r.boxes.cls.cpu().numpy().astype(int):
                        class_counts[model.names[cls_id]] += 1

    return {
        'model_name': model_name,
        'avg_time_ms': np.mean(times),
        'std_time_ms': np.std(times),
        'fps': 1000 / np.mean(times),
        'avg_detections': np.mean(detections),
        'avg_confidence': np.mean(confidences) if confidences else 0,
        'class_counts': dict(class_counts),
        'total_detections': sum(class_counts.values())
    }

In [ ]:
# Get test images from Road Lane dataset
test_images_lane = list((DATASETS['road_lane']['path'] / 'test' / 'images').glob('*.jpg'))[:20]
print(f"Road Lane test images: {len(test_images_lane)}")

# Get test images from BDD100K
bdd_val_dir = DATASETS['bdd100k']['path'] / 'bdd100k' / 'images' / '100k' / 'val'
test_images_bdd = list(bdd_val_dir.glob('*.jpg'))[:50] if bdd_val_dir.exists() else []
print(f"BDD100K test images: {len(test_images_bdd)}")

In [ ]:
# Run benchmarks on Road Lane test set
print("Benchmarking on Road Lane Test Set")

benchmark_results = {}

if test_images_lane:
    # Benchmark fine-tuned models
    for name, info in FINETUNED_MODELS.items():
        if 'RoadLane' in name:  # Only Road Lane models
            print(f"\nBenchmarking {name}...")
            benchmark_results[name] = benchmark_inference(
                info['model'], test_images_lane, name, num_runs=2
            )

    # Benchmark pre-trained models for comparison
    for name, model in PRETRAINED_MODELS.items():
        print(f"\nBenchmarking {name}...")
        benchmark_results[name] = benchmark_inference(
            model, test_images_lane, name, num_runs=2
        )

In [ ]:
print("\nPERFORMANCE COMPARISON")

comparison_data = []
for name, result in benchmark_results.items():
    comparison_data.append({
        'Model': name,
        'Avg Time (ms)': f"{result['avg_time_ms']:.1f}",
        'FPS': f"{result['fps']:.1f}",
        'Avg Detections': f"{result['avg_detections']:.1f}",
        'Avg Confidence': f"{result['avg_confidence']:.2%}",
        'Total Detections': result['total_detections']
    })

df_results = pd.DataFrame(comparison_data)
print(df_results.to_string(index=False))

In [ ]:
# Visualize performance comparison
if benchmark_results:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    models = list(benchmark_results.keys())
    colors = plt.cm.Set2(np.linspace(0, 1, len(models)))

    # 1. Inference Time
    ax = axes[0, 0]
    times = [benchmark_results[m]['avg_time_ms'] for m in models]
    bars = ax.bar(models, times, color=colors, edgecolor='black')
    ax.set_ylabel('Time (ms)')
    ax.set_title('⏱️ Inference Time (lower is better)', fontweight='bold')
    ax.tick_params(axis='x', rotation=45)
    for bar, t in zip(bars, times):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'{t:.1f}', ha='center', va='bottom', fontsize=9)

    # 2. FPS
    ax = axes[0, 1]
    fps = [benchmark_results[m]['fps'] for m in models]
    bars = ax.bar(models, fps, color=colors, edgecolor='black')
    ax.set_ylabel('FPS')
    ax.set_title('🚀 Frames Per Second (higher is better)', fontweight='bold')
    ax.tick_params(axis='x', rotation=45)
    for bar, f in zip(bars, fps):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'{f:.1f}', ha='center', va='bottom', fontsize=9)

    # 3. Average Detections
    ax = axes[1, 0]
    dets = [benchmark_results[m]['avg_detections'] for m in models]
    bars = ax.bar(models, dets, color=colors, edgecolor='black')
    ax.set_ylabel('Detections')
    ax.set_title('📦 Avg Detections per Image', fontweight='bold')
    ax.tick_params(axis='x', rotation=45)

    # 4. Average Confidence
    ax = axes[1, 1]
    confs = [benchmark_results[m]['avg_confidence'] for m in models]
    bars = ax.bar(models, confs, color=colors, edgecolor='black')
    ax.set_ylabel('Confidence')
    ax.set_title('🎯 Average Confidence Score', fontweight='bold')
    ax.tick_params(axis='x', rotation=45)
    ax.set_ylim(0, 1)

    plt.tight_layout()
    plt.show()

## **Detection quality test**

In [ ]:
def analyze_detection_relevance(model, images, model_name):
    """
    Analyze what classes the model detects and categorize them.
    """
    traffic_detections = Counter()
    non_traffic_detections = Counter()
    other_detections = Counter()

    for img in images:
        results = model(img, verbose=False)
        for r in results:
            for cls_id in r.boxes.cls.cpu().numpy().astype(int):
                class_name = model.names[cls_id]
                if class_name.lower() in {c.lower() for c in TRAFFIC_CLASSES}:
                    traffic_detections[class_name] += 1
                elif class_name.lower() in {c.lower() for c in NON_TRAFFIC_CLASSES}:
                    non_traffic_detections[class_name] += 1
                else:
                    other_detections[class_name] += 1

    total = sum(traffic_detections.values()) + sum(non_traffic_detections.values()) + sum(other_detections.values())

    return {
        'model_name': model_name,
        'traffic_detections': dict(traffic_detections),
        'non_traffic_detections': dict(non_traffic_detections),
        'other_detections': dict(other_detections),
        'traffic_count': sum(traffic_detections.values()),
        'non_traffic_count': sum(non_traffic_detections.values()),
        'other_count': sum(other_detections.values()),
        'total': total,
        'traffic_ratio': sum(traffic_detections.values()) / max(total, 1)
    }

In [ ]:
TRAFFIC_CLASSES = {
    'car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle',
    'person', 'traffic light', 'stop sign', 'parking meter',
    'lane', 'drivable area', 'road', 'rider', 'motor', 'bike',
    'divider-line', 'dotted-line', 'double-line', 'random-line',
    'road-sign-line', 'solid-line', 'traffic sign'
}

NON_TRAFFIC_CLASSES = {
    'toothbrush', 'hair drier', 'wine glass', 'cup', 'fork', 'knife',
    'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',
    'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'couch', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
    'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
    'baseball glove', 'skateboard', 'surfboard', 'tennis racket'
}

print(f"Traffic-related classes: {len(TRAFFIC_CLASSES)}")
print(f"Non-traffic classes: {len(NON_TRAFFIC_CLASSES)}")

In [ ]:
relevance_results = {}

# Use Road Lane test images
test_images = test_images_lane[:15] if test_images_lane else []

if test_images:
    # Analyze fine-tuned models
    for name, info in FINETUNED_MODELS.items():
        print(f"\nAnalyzing {name}...")
        relevance_results[name] = analyze_detection_relevance(
            info['model'], test_images, name
        )

    # Analyze pre-trained models
    for name, model in PRETRAINED_MODELS.items():
        print(f"\nAnalyzing {name}...")
        relevance_results[name] = analyze_detection_relevance(
            model, test_images, name
        )

In [ ]:
# Display relevance results
print("\nDETECTION RELEVANCE SUMMARY")

for name, result in relevance_results.items():
    print(f"\n{name}")
    print(f"   Traffic-related detections: {result['traffic_count']}")
    print(f"   Non-traffic detections: {result['non_traffic_count']}")
    print(f"   Other detections: {result['other_count']}")
    print(f"   Traffic Focus Ratio: {result['traffic_ratio']:.1%}")

    if result['traffic_detections']:
        top_traffic = sorted(result['traffic_detections'].items(), key=lambda x: x[1], reverse=True)[:5]
        print(f"   Top traffic classes: {dict(top_traffic)}")

    if result['non_traffic_detections']:
        print(f"   Unwanted detections: {result['non_traffic_detections']}")

In [ ]:
# Visualize detection relevance comparison
if relevance_results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    models = list(relevance_results.keys())

    # Stacked bar chart: Traffic vs Non-Traffic detections
    ax = axes[0]
    traffic = [relevance_results[m]['traffic_count'] for m in models]
    non_traffic = [relevance_results[m]['non_traffic_count'] for m in models]
    other = [relevance_results[m]['other_count'] for m in models]

    x = np.arange(len(models))
    width = 0.6

    ax.bar(x, traffic, width, label='Traffic', color='#2ecc71')
    ax.bar(x, non_traffic, width, bottom=traffic, label='Non-Traffic', color='#e74c3c')
    ax.bar(x, other, width, bottom=[t+n for t,n in zip(traffic, non_traffic)],
           label='Other', color='#95a5a6')

    ax.set_ylabel('Number of Detections')
    ax.set_title('🚗 Detection Categories by Model', fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=45, ha='right')
    ax.legend()

    # Traffic Focus Ratio
    ax = axes[1]
    ratios = [relevance_results[m]['traffic_ratio'] * 100 for m in models]
    colors = ['#2ecc71' if r > 80 else '#f39c12' if r > 50 else '#e74c3c' for r in ratios]
    bars = ax.bar(models, ratios, color=colors, edgecolor='black')
    ax.set_ylabel('Traffic Focus Ratio (%)')
    ax.set_title('Traffic Detection Focus (higher = better)', fontweight='bold')
    ax.set_ylim(0, 100)
    ax.axhline(y=80, color='green', linestyle='--', alpha=0.7, label='Good (80%)')
    ax.tick_params(axis='x', rotation=45)

    for bar, ratio in zip(bars, ratios):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
                f'{ratio:.0f}%', ha='center', fontweight='bold')

    plt.tight_layout()
    plt.show()

**Visual comparison: pre-trained vs fine-tuned**

In [ ]:
def compare_detections(image_path, models_dict, title="Detection Comparison"):
    """
    Compare detection results from multiple models on the same image.
    """
    n_models = len(models_dict)
    fig, axes = plt.subplots(1, n_models + 1, figsize=(5 * (n_models + 1), 5))

    # Original image
    img = cv2.imread(str(image_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[0].imshow(img_rgb)
    axes[0].set_title('Original', fontsize=11)
    axes[0].axis('off')

    # Detection results from each model
    for idx, (name, model) in enumerate(models_dict.items(), 1):
        results = model(image_path, verbose=False)
        annotated = results[0].plot()
        annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)

        n_det = len(results[0].boxes)

        # Get detected classes
        detected_classes = []
        if n_det > 0:
            for cls_id in results[0].boxes.cls.cpu().numpy().astype(int):
                detected_classes.append(model.names[cls_id])

        axes[idx].imshow(annotated_rgb)
        axes[idx].set_title(f'{name}\n({n_det} detections)', fontsize=10)
        axes[idx].axis('off')

    plt.suptitle(title, fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:
# Compare models on sample images
print("🎨 Visual Detection Comparison")
print("=" * 70)

# Combine all models for comparison
all_models = {}

# Add fine-tuned models
for name, info in FINETUNED_MODELS.items():
    all_models[name] = info['model']

# Add pre-trained models
all_models.update(PRETRAINED_MODELS)

# Show comparison on test images
if test_images_lane:
    for i, img_path in enumerate(test_images_lane[:3]):
        print(f"\n📷 Image {i+1}: {img_path.name}")
        compare_detections(img_path, all_models, f"Sample {i+1}: {img_path.name}")

In [ ]:
# Final summary
print("\n" + "=" * 70)
print("📋 FINE-TUNING SUMMARY")
print("=" * 70)

print("\n🏆 TRAINED MODELS:")
for name, info in FINETUNED_MODELS.items():
    print(f"   ✅ {name}")
    print(f"      Classes: {len(info['classes'])}")
    print(f"      Path: {info['path']}")

print("\n📊 PERFORMANCE HIGHLIGHTS:")
if benchmark_results:
    # Find fastest and most accurate
    fastest = min(benchmark_results.items(), key=lambda x: x[1]['avg_time_ms'])
    most_detections = max(benchmark_results.items(), key=lambda x: x[1]['avg_detections'])

    print(f"   🚀 Fastest: {fastest[0]} ({fastest[1]['fps']:.1f} FPS)")
    print(f"   📦 Most detections: {most_detections[0]} ({most_detections[1]['avg_detections']:.1f} per image)")

print("\n🎯 DETECTION FOCUS:")
if relevance_results:
    for name, result in relevance_results.items():
        status = "✅" if result['traffic_ratio'] > 0.8 else "⚠️" if result['traffic_ratio'] > 0.5 else "❌"
        print(f"   {status} {name}: {result['traffic_ratio']:.1%} traffic-focused")

print("\n💡 RECOMMENDATIONS:")
print("""
   1. Use fine-tuned models for traffic detection - they're focused on
      relevant objects and won't waste resources on irrelevant items.

   2. YOLO11n is best for real-time applications (faster but less accurate)

   3. YOLO11l is best when accuracy is critical (slower but more accurate)

   4. The Road Lane model is specialized for lane detection

   5. The BDD100K model covers broader traffic scenarios
""")

print("=" * 70)
print("✅ Fine-tuning Complete!")
print("=" * 70)